# Analysis of ENSO predictability

In [1]:
import re
import os
import glob
import doppyo
import numpy as np
import pandas as pd
import xarray as xr
from scipy.ndimage.measurements import label

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

# Notebook specific -----
%matplotlib inline

In [2]:
saveloc = '/OSM/CBR/OA_DCFP/work/squ027/squire_scratch/projects/papers/Risbey_Nature/figures'

# Notebook-specific functions

In [3]:
lambda_anomalize = lambda data, clim: doppyo.utils.datetime_to_leadtime(
                                          doppyo.utils.anomalize(
                                              doppyo.utils.leadtime_to_datetime(data), clim))

In [4]:
def natural_keys(path):
    basename = os.path.basename(path)
    return [ int(c) for c in re.split('(\d+)', basename) if c.isdigit()]

In [5]:
def cv_anomalize(fcst, clim_ts, time_dim = 'init_date'):
    """ Anomalize provided data in a "cross-validated" manner """
    
    fcst_dates = fcst[time_dim].values
    if not hasattr(fcst_dates, "__iter__"):
        fcst_dates = [fcst_dates]
    clim_dates = clim_ts[time_dim].values
    if not hasattr(clim_dates, "__iter__"):
        clim_dates = [clim_dates]
        
    clim = clim_ts.sel({time_dim : sorted(set(clim_dates) - set(fcst_dates))}) \
                  .groupby(time_dim+'.month').mean(time_dim)

    return fcst - clim.sel(month = fcst[time_dim].dt.month.values)

In [6]:
def Delsole_regression(da_predict,  da_train):
    """ 
        Build model over dates in da_predict using data in da_train
        Overlapping periods in da_predict and da_train are excluded from the training period
    """
    
    def _fit(da):
        return doppyo.utils.polyfit(da.sel(lead_time=0), da.sel(lead_time=range(1,12)), order=1, over_dims='init_date')
    
    def _predict(da, p):
        month = da.init_date.dt.month.values[0]
        return doppyo.utils.polyval(da, p.sel(month=month, drop=True), over_dims='init_date')

    if da_predict.shape == ():
        da_predict = da_predict.expand_dims('init_date')
    da_predict = doppyo.utils.prune(da_predict)

    train_dates = da_train['init_date'].values
    if not hasattr(train_dates, "__iter__"):
        train_dates = [train_dates]
    predict_dates = da_predict['init_date'].values
    if not hasattr(predict_dates, "__iter__"):
        predict_dates = [predict_dates]
        
    da_use = da_train.sel({'init_date' : sorted(set(train_dates) - set(predict_dates))})
    
    p = da_use.groupby('init_date.month').apply(_fit)
    
    if len(predict_dates) == 1:
        return _predict(da_predict, p=p).squeeze()
    else:
        return da_predict.groupby('init_date.month', squeeze=True).apply(_predict, p=p).drop('month')

In [7]:
def where_elninos(da):
    
    s = np.array([[False,True,True],[True,True,True],[True,True,False]])
    
    where_elnino = da.rolling(init_date=3, center=True).mean() > 0.5
    where_elnino.values = label(1*where_elnino, structure=s)[0]
    
    return where_elnino

In [8]:
def where_laninas(da):
    
    s = np.array([[False,True,True],[True,True,True],[True,True,False]])
    
    where_lanina = da.rolling(init_date=3, center=True).mean() < -0.5
    where_lanina.values = label(1*where_lanina, structure=s)[0]
    
    return where_lanina

In [9]:
def where_condition_events(where_event, method):

    for event_id in range(1, where_event.max().values+1):
        specific_event = 1*(where_event == event_id)

        if method == 'decay':
            specific_event = -1 * (specific_event - 1)

        int_event = doppyo.utils.integrate(specific_event, over_dim='lead_time', 
                                            x=(1+0*specific_event.lead_time.astype(int)).cumsum('lead_time'), 
                                            method='rect', cumulative=True)
        int_cmpar = (1+0*specific_event.astype(int)).cumsum('lead_time')

        specific_event_conditioned = specific_event.where(int_event != int_cmpar, other=0)
        
        if method == 'decay':
            specific_event_conditioned = specific_event_conditioned.where(specific_event.sel(lead_time=0) == 0, other=0)
            
        if event_id == 1:
            event_conditioned = specific_event_conditioned
        else:
            event_conditioned = event_conditioned + specific_event_conditioned
            
    return event_conditioned

In [10]:
def plot_fcst(t, x, ax=None, cmap='jet', **kwargs):
    # Convert dates to numbers first ---- 
    try:
        inxval = matplotlib.dates.date2num(t.to_index().to_pydatetime())
        points = np.array([inxval, x.values]).T.reshape(-1,1,2)
    except:
        inxval = matplotlib.dates.date2num(t)
        points = np.array([inxval, x]).T.reshape(-1,1,2)
    segments = np.concatenate([points[:-1],points[1:]], axis=1)
    
    lc = LineCollection(segments, cmap=cmap, **kwargs)
    lc.set_array(inxval)
    
    monthFmt = matplotlib.dates.DateFormatter("%Y")
    if ax is None:
        ax = plt.gca()
        
    ax.add_collection(lc)
    ax.xaxis.set_major_formatter(monthFmt)
    ax.autoscale_view()
    ax.xaxis_date()

In [11]:
colors1 = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69']
colors2 = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628']
colors3 = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f']
colors4 = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']
colorsd = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2']
colors = colorsd

# A note about climatologies and bias corrections
The CMC Can3/4 hindcasts / forecasts span the periods 1981-01 -> 2010-12 / 2011-01 -> 2018-09. The GFDL and COLA models span 1982-01 -> 2018-11. The Hadley ISST data spans 1870-01 -> 2018-02.
The mutual span of these datasets is **1982-01 -> 2018-02**

In this notebook we compare and contrast a number of methods for computing anomalies and for bias correcting data. In all cases, **17 years of data are use to build climatologies**. The methods are abbreviated as follows:

**c0** : model anomalies over the period **1999-01 -> 2015-12** are computed relative to the observed climatology over the period **1982-01 -> 1998-12** - see Saha et al. (2006), Kirtman (2003), Kirtman et al. (1997), Kirtman and Min (2009)... Observed anomalies are computed in the same way.

**c1** : model anomalies over the period **1999-01 -> 2015-12** are computed relative to the (lead-time-dependent) ensemble mean model climatology over the same period using cross-validation - see Kirtman and Min (2009), Kirtman *et al.* (2014)... Similarly, observed anomalies over the period **1999-01 -> 2015-12** are computed relative to the observed climatology over the same period using cross-validation

**c2** : model anomalies over the period **1999-01 -> 2015-12** are computed relative to the (lead-time-dependent) ensemble mean model climatology over the period **1982-01 -> 1998-12**. Similarly, observed anomalies over the period **1999-01 -> 2015-12** are computed relative to the observed climatology over the period **1982-01 -> 1998-12**

**c3** : as in Delsole and Tippet, the (lead-time-dependent) mean forecast error for each calendar month using hindcasts whose verifications lie within the period 1982–98 inclusive. This error is then subtracted from the **c0** anomalies. This approach is subtly different from **c2** - if instead, the mean error were computed as the difference between the lead-time-dependent model climatology and the observed climatology over the period 1982-1998, then **c3** -> **c2**. Observed anomalies are the same as for **c2**.

**Note**, in actuality model anomalies are computed over the full model period, but they should only be examined for **1999-01 -> 2015-12**

For comparison, we also compute:

**c1full** : model anomalies over the period **1982-01 -> 2015-12** are computed relative to the ensemble mean, lead=0 model climatology over the same period using cross-validation - see Kirtman and Min (2009), Kirtman *et al.* (2014)... Similarly, observed anomalies over the period **1982-01 -> 2015-12** are computed relative to the observed climatology over the same period using cross-validation

**c1ncv** : model anomalies over the period **1982-01 -> 2015-12** are computed relative to the ensemble mean, lead=0 model climatology over the same period **without cross-validation**. Similarly, observed anomalies over the period **1982-01 -> 2015-12** are computed relative to the observed climatology over the same period **without cross-validation**

# Load the data

In [12]:
compute_data = False # If True, recomputes quantities from raw data and saves to dataloc, otherwise loads from dataloc
dataloc = '/OSM/CBR/OA_DCFP/work/squ027/intermediate_products/tmp/'

### Raw NOAA OISST data (downloaded from http://www.cpc.ncep.noaa.gov/data/indices/sstoi.indices.)

In [13]:
time = pd.date_range(start='1982-01',end='2019-02',freq='MS')
data = np.array([26.72,26.7,27.2,28.02,28.54,28.75,28.1,27.93,28.11,28.64,28.81,29.21,29.36,29.13,29.03,28.91,28.89,28.24,27.07,26.53,26.44,25.87,25.58,25.59,25.64,26.39,26.86,27.39,27.39,26.86,26.74,26.34,26.43,25.93,25.41,25,25.43,25.67,26.23,26.8,27.11,26.86,26.69,26.5,26.25,26.19,26.19,26.11,25.79,25.94,26.65,27.44,27.5,27.69,27.37,27.15,27.33,27.57,27.73,27.7,27.91,28.02,28.47,28.8,28.75,29.03,28.8,28.58,28.39,28.07,27.99,27.6,27.32,27.22,27.31,27.32,26.48,26.11,25.57,25.24,25.43,24.62,24.27,24.33,24.53,25.33,25.9,26.69,27.09,26.98,26.74,26.33,26.25,26.26,26.24,26.38,26.55,26.95,27.46,28.02,28.06,27.58,27.25,27.05,26.75,26.98,26.72,26.91,27.01,26.93,27.25,27.98,28.35,28.36,27.92,27.44,27.07,27.63,27.86,28.37,28.41,28.63,28.83,29.14,28.99,28.02,27.53,26.64,26.48,26.34,26.51,26.73,26.69,26.97,27.66,28.59,28.82,28.28,27.55,26.84,26.92,26.93,26.91,26.76,26.6,26.59,27.27,27.9,28.04,27.99,27.35,27.35,27,27.49,27.87,27.87,27.55,27.45,27.63,27.93,27.73,27.59,27.01,26.33,25.96,25.67,25.66,25.57,25.74,25.85,26.62,27.36,27.37,27.32,27.09,26.56,26.35,26.24,26.19,26.02,25.96,26.36,27.03,28.03,28.6,28.94,28.92,28.84,28.93,29.23,29.32,29.26,29.1,28.86,28.67,28.56,28.47,26.72,25.94,25.49,25.61,25.34,25.18,24.79,24.9,25.41,26.25,26.84,26.97,26.6,26.35,25.59,25.71,25.64,25.12,24.9,24.65,25.19,26.08,27.01,27.12,27.03,26.72,26.45,26.21,25.96,25.78,25.59,25.74,26.11,26.84,27.52,27.6,27.68,27.32,26.87,26.55,26.59,26.45,26.17,26.5,26.95,27.32,27.94,28.15,28.43,27.98,27.79,27.83,28.05,28.27,28.09,27.76,27.49,27.81,27.81,27.37,27.48,27.43,26.85,26.96,27.19,27.05,26.89,26.74,26.86,27.1,27.84,28.06,27.76,27.69,27.54,27.47,27.38,27.31,27.31,27.1,26.96,27.55,28.07,28.2,28.05,27.47,26.88,26.63,26.75,26.34,25.89,25.64,26.08,26.57,27.59,27.91,27.85,27.35,27.22,27.34,27.47,27.73,27.76,27.26,26.81,27.18,27.78,27.57,27.55,26.79,26.2,25.77,25.22,25.06,24.97,24.71,24.83,26.07,26.83,27.18,27.17,27.19,26.85,26.44,26.33,26.3,25.74,25.54,26.04,26.67,27.5,28.03,28.11,27.94,27.53,27.47,27.63,28.19,28.3,28.07,27.94,28.29,28.36,27.68,27,26.09,25.5,25.07,25.01,25.07,24.95,24.93,25.46,26.23,27.02,27.42,27.46,26.96,26.19,25.98,25.72,25.6,25.53,25.49,26.03,26.63,27.38,27.8,27.95,27.75,27.55,27.24,26.98,27.01,26.46,26.16,26.32,27,27.68,27.57,27.43,26.91,26.54,26.65,26.36,26.65,26.53,26.06,26.18,26.99,28.01,28.31,28.11,27.4,27.02,27.17,27.17,27.5,27.35,27.1,27.29,27.79,28.56,28.88,28.96,28.82,28.89,29,29.15,29.6,29.39,29.17,29.12,28.9,28.87,28.15,27.53,26.73,26.28,26.11,25.96,26.1,26.16,26.25,26.87,27.34,28.1,28.3,28.19,27.61,26.67,26.29,26.23,25.79,25.8,25.82,25.83,26.48,27.42,27.72,27.85,27.52,27.11,27.1,27.55,27.64,27.53,27.08,27.38])

oisst_nino34_full_ts = xr.DataArray(data, coords=[('time', time)])
oisst_nino34_full = doppyo.utils.stack_by_init_date(oisst_nino34_full_ts, init_dates=pd.date_range('1982-01','2015-12',freq='MS'), N_lead_steps=12).rename('full')

##### c0 anomalies

In [14]:
if compute_data:
    clim_period = slice('1982','1998')
    oisst_nino34_c0_ts = (oisst_nino34_full_ts.groupby('time.month') - oisst_nino34_full_ts.sel(time=clim_period).groupby('time.month').mean('time')).drop('month')
    oisst_nino34_c0 = doppyo.utils.stack_by_init_date(oisst_nino34_c0_ts, init_dates=pd.date_range('1982-01','2015-12',freq='MS'), N_lead_steps=12).rename('c0')
    
    oisst_nino34_c0.to_netcdf(dataloc + 'ENSO_predictability.oisst_nino34_c0.nc')
else:
    oisst_nino34_c0 = xr.open_dataset(dataloc + 'ENSO_predictability.oisst_nino34_c0.nc')['c0']

OSError: [Errno -51] NetCDF: Unknown file format: b'/OSM/CBR/OA_DCFP/work/squ027/intermediate_products/tmp/ENSO_predictability.oisst_nino34_c0.nc'

##### c1 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')
    clim_use = oisst_nino34_full_ts.sel(time=clim_period)
    oisst_nino34_c1_ts = oisst_nino34_full_ts.groupby('time').apply(cv_anomalize, clim_ts=clim_use, time_dim='time')
    oisst_nino34_c1 = doppyo.utils.stack_by_init_date(oisst_nino34_c1_ts, init_dates=pd.date_range('1982-01','2018-02',freq='MS'), N_lead_steps=12).rename('c1').drop('month')
    
    oisst_nino34_c1.to_netcdf(dataloc + 'ENSO_predictability.oisst_nino34_c1.nc')
else:
    oisst_nino34_c1 = xr.open_dataset(dataloc + 'ENSO_predictability.oisst_nino34_c1.nc')['c1']

##### c2 anomalies

In [ ]:
if compute_data:
    oisst_nino34_c2_ts = oisst_nino34_c0_ts
    oisst_nino34_c2 = oisst_nino34_c0.rename('c2')
    
    oisst_nino34_c2.to_netcdf(dataloc + 'ENSO_predictability.oisst_nino34_c2.nc')
else:
    oisst_nino34_c2 = xr.open_dataset(dataloc + 'ENSO_predictability.oisst_nino34_c2.nc')['c2']

##### c3 anomalies

In [ ]:
if compute_data:
    oisst_nino34_c3_ts = oisst_nino34_c0_ts
    oisst_nino34_c3 = oisst_nino34_c0.rename('c3')
    
    oisst_nino34_c3.to_netcdf(dataloc + 'ENSO_predictability.oisst_nino34_c3.nc')
else:
    oisst_nino34_c3 = xr.open_dataset(dataloc + 'ENSO_predictability.oisst_nino34_c3.nc')['c3']

##### c1 full anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','2015')
    clim_use = oisst_nino34_full_ts.sel(time=clim_period)
    oisst_nino34_c1full_ts = oisst_nino34_full_ts.groupby('time').apply(cv_anomalize, clim_ts=clim_use, time_dim='time')
    oisst_nino34_c1full = doppyo.utils.stack_by_init_date(oisst_nino34_c1full_ts, init_dates=pd.date_range('1982-01','2018-02',freq='MS'), N_lead_steps=12).rename('c1full').drop('month')
    
    oisst_nino34_c1full.to_netcdf(dataloc + 'ENSO_predictability.oisst_nino34_c1full.nc')
else:
    oisst_nino34_c1full = xr.open_dataset(dataloc + 'ENSO_predictability.oisst_nino34_c1full.nc')['c1full']

##### c1 ncv anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')
    oisst_nino34_c1ncv_ts = (oisst_nino34_full_ts.groupby('time.month') - oisst_nino34_full_ts.sel(time=clim_period).groupby('time.month').mean('time')).drop('month')
    oisst_nino34_c1ncv = doppyo.utils.stack_by_init_date(oisst_nino34_c1ncv_ts, init_dates=pd.date_range('1982-01','2015-12',freq='MS'), N_lead_steps=12).rename('c1ncv')
    
    oisst_nino34_c1ncv.to_netcdf(dataloc + 'ENSO_predictability.oisst_nino34_c1ncv.nc')
else:
    oisst_nino34_c1ncv = xr.open_dataset(dataloc + 'ENSO_predictability.oisst_nino34_c1ncv.nc')['c1ncv']

##### Combine into dataset

In [ ]:
oisst_nino34 = oisst_nino34_full.to_dataset()
oisst_nino34['c0'] = oisst_nino34_c0
oisst_nino34['c1'] = oisst_nino34_c1
oisst_nino34['c2'] = oisst_nino34_c2
oisst_nino34['c3'] = oisst_nino34_c3
oisst_nino34['c1full'] = oisst_nino34_c1full
oisst_nino34['c1ncv'] = oisst_nino34_c1ncv

### Raw Hadley ISST data

In [ ]:
if compute_data:
    had_sst_full_ts = xr.open_dataset('/OSM/CBR/OA_DCFP/work/squ027/intermediate_products/tmp/ENSO_predictability.had_sst_raw_ts.nc')['sst'].rename('hadsst')
    had_nino34_full_ts = doppyo.diagnostic.nino34(had_sst_full_ts).rename('full_ts')
    had_nino34_full = doppyo.utils.stack_by_init_date(had_nino34_full_ts, init_dates=pd.date_range('1982-01','2015-12',freq='MS'), N_lead_steps=12).rename('full')
    
    had_nino34_full_ts.to_netcdf(dataloc + 'ENSO_predictability.had_nino34_full_ts.nc')
    had_nino34_full.to_netcdf(dataloc + 'ENSO_predictability.had_nino34_full.nc')
else:
    had_nino34_full_ts = xr.open_dataset(dataloc + 'ENSO_predictability.had_nino34_full_ts.nc')['full_ts']
    had_nino34_full = xr.open_dataset(dataloc + 'ENSO_predictability.had_nino34_full.nc')['full']

##### c0 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    had_nino34_c0_ts = (had_nino34_full_ts.groupby('time.month') - had_nino34_full_ts.sel(time=clim_period).groupby('time.month').mean('time')).drop('month')
    had_nino34_c0 = doppyo.utils.stack_by_init_date(had_nino34_c0_ts, init_dates=pd.date_range('1982-01','2015-12',freq='MS'), N_lead_steps=12).rename('c0')
    
    had_nino34_c0.to_netcdf(dataloc + 'ENSO_predictability.had_nino34_c0.nc')
else:
    had_nino34_c0 = xr.open_dataset(dataloc + 'ENSO_predictability.had_nino34_c0.nc')['c0']

##### c1 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')
    clim_use = had_nino34_full_ts.sel(time=clim_period)
    had_nino34_c1_ts = had_nino34_full_ts.groupby('time').apply(cv_anomalize, clim_ts=clim_use, time_dim='time')
    had_nino34_c1 = doppyo.utils.stack_by_init_date(had_nino34_c1_ts, init_dates=pd.date_range('1982-01','2018-02',freq='MS'), N_lead_steps=12).rename('c1').drop('month')
    
    had_nino34_c1.to_netcdf(dataloc + 'ENSO_predictability.had_nino34_c1.nc')
else:
    had_nino34_c1 = xr.open_dataset(dataloc + 'ENSO_predictability.had_nino34_c1.nc')['c1']

##### c2 anomalies

In [ ]:
if compute_data:
    had_nino34_c2_ts = had_nino34_c0_ts
    had_nino34_c2 = had_nino34_c0.rename('c2')
    
    had_nino34_c2.to_netcdf(dataloc + 'ENSO_predictability.had_nino34_c2.nc')
else:
    had_nino34_c2 = xr.open_dataset(dataloc + 'ENSO_predictability.had_nino34_c2.nc')['c2']

##### c3 anomalies

In [ ]:
if compute_data:
    had_nino34_c3_ts = had_nino34_c0_ts
    had_nino34_c3 = had_nino34_c0.rename('c3')
    
    had_nino34_c3.to_netcdf(dataloc + 'ENSO_predictability.had_nino34_c3.nc')
else:
    had_nino34_c3 = xr.open_dataset(dataloc + 'ENSO_predictability.had_nino34_c3.nc')['c3']

##### c1 full anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','2015')
    clim_use = had_nino34_full_ts.sel(time=clim_period)
    had_nino34_c1full_ts = had_nino34_full_ts.groupby('time').apply(cv_anomalize, clim_ts=clim_use, time_dim='time')
    had_nino34_c1full = doppyo.utils.stack_by_init_date(had_nino34_c1full_ts, init_dates=pd.date_range('1982-01','2018-02',freq='MS'), N_lead_steps=12).rename('c1full').drop('month')
    
    had_nino34_c1full.to_netcdf(dataloc + 'ENSO_predictability.had_nino34_c1full.nc')
else:
    had_nino34_c1full = xr.open_dataset(dataloc + 'ENSO_predictability.had_nino34_c1full.nc')['c1full']

##### c1 ncv anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')
    had_nino34_c1ncv_ts = (had_nino34_full_ts.groupby('time.month') - had_nino34_full_ts.sel(time=clim_period).groupby('time.month').mean('time')).drop('month')
    had_nino34_c1ncv = doppyo.utils.stack_by_init_date(had_nino34_c1ncv_ts, init_dates=pd.date_range('1982-01','2015-12',freq='MS'), N_lead_steps=12).rename('c1ncv')
    
    had_nino34_c1ncv.to_netcdf(dataloc + 'ENSO_predictability.had_nino34_c1ncv.nc')
else:
    had_nino34_c1ncv = xr.open_dataset(dataloc + 'ENSO_predictability.had_nino34_c1ncv.nc')['c1ncv']

##### Combine into dataset

In [ ]:
had_nino34 = had_nino34_full.to_dataset()
had_nino34['c0'] = had_nino34_c0
had_nino34['c1'] = had_nino34_c1
had_nino34['c2'] = had_nino34_c2
had_nino34['c3'] = had_nino34_c3
had_nino34['c1full'] = had_nino34_c1full
had_nino34['c1ncv'] = had_nino34_c1ncv

# Choose which obs to use

In [ ]:
obs_2_use = oisst_nino34 # had_nino34

### Raw COLA-RSMAS-CCSM4 data

In [ ]:
if compute_data:
    filelist = glob.glob('/OSM/CBR/OA_DCFP/data2/model_output/NMME/phase1/COLA-RSMAS-CCSM4/monthly/sst_mon_COLA-RSMAS-CCSM4_198201_r*i1p1_198201-201811.nc')
    filelist.sort(key=natural_keys)

    cola_sst_raw = xr.open_mfdataset(filelist, decode_times=False)['sst'] \
                     .rename({'X' : 'lon', 'Y' : 'lat', 'M' : 'ensemble', 
                              'L' : 'lead_time', 'S' : 'init_date'}).compute()

    basedate = np.datetime64(cola_sst_raw.init_date.units.split(' ')[2])
    cola_sst_raw['init_date'] = np.array([doppyo.sugar.month_delta(basedate, int(shift)) for shift in cola_sst_raw.init_date.values])

    cola_sst_raw['lead_time'] = [int(lead_time - 0.5) for lead_time in cola_sst_raw['lead_time']]
    cola_sst_raw.lead_time.attrs['units'] = 'MS'

    cola_sst_raw['ensemble'] = [int(ensemble) for ensemble in cola_sst_raw['ensemble']]

##### Compute nino3.4

In [ ]:
if compute_data:
    cola_nino34_full = doppyo.diagnostic.nino34(cola_sst_raw).rename('full')
    
    cola_nino34_full.to_netcdf(dataloc + 'ENSO_predictability.cola_nino34_full.nc')
else:
    cola_nino34_full = xr.open_dataset(dataloc + 'ENSO_predictability.cola_nino34_full.nc')['full']

##### c0 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    clim = obs_2_use['full'].sel(init_date=clim_period, lead_time=0).groupby('init_date.month').mean('init_date').drop('lead_time')
    cola_nino34_c0 = cola_nino34_full.groupby('init_date').apply(lambda_anomalize, clim=clim).rename('c0')
    
    cola_nino34_c0.to_netcdf(dataloc + 'ENSO_predictability.cola_nino34_c0.nc')
else:
    cola_nino34_c0 = xr.open_dataset(dataloc + 'ENSO_predictability.cola_nino34_c0.nc')['c0']

##### c1 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')
    cola_nino34_full_use = cola_nino34_full.sel(init_date=clim_period).mean('ensemble')
    cola_nino34_c1 = cola_nino34_full.groupby('init_date').apply(cv_anomalize, clim_ts=cola_nino34_full_use).rename('c1')
    
    cola_nino34_c1.to_netcdf(dataloc + 'ENSO_predictability.cola_nino34_c1.nc')
else:
    cola_nino34_c1 = xr.open_dataset(dataloc + 'ENSO_predictability.cola_nino34_c1.nc')['c1']

##### c2 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')

    clim = cola_nino34_full.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    cola_nino34_c2 = (cola_nino34_full.groupby('init_date.month') - clim).rename('c2')
    
    cola_nino34_c2.to_netcdf(dataloc + 'ENSO_predictability.cola_nino34_c2.nc')
else:
    cola_nino34_c2 = xr.open_dataset(dataloc + 'ENSO_predictability.cola_nino34_c2.nc')['c2']

##### c3 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    
    init_dates = cola_nino34_c0.sel(init_date=clim_period).init_date.copy()
    lead_times = cola_nino34_c0.sel(init_date=clim_period).lead_time.copy()
    init_dates.values = np.arange(len(init_dates))
    mask = (init_dates + lead_times) <= init_dates[-1]

    fcst_er = (cola_nino34_c0.where(mask) - obs_2_use['c0'].where(mask)).groupby('init_date.month').mean(['init_date','ensemble'])
    
    cola_nino34_c3 = (cola_nino34_c0.groupby('init_date.month') - fcst_er).rename('c3')

    cola_nino34_c3.to_netcdf(dataloc + 'ENSO_predictability.cola_nino34_c3.nc')
else:
    cola_nino34_c3 = xr.open_dataset(dataloc + 'ENSO_predictability.cola_nino34_c3.nc')['c3']

##### The **c2** anomalies are similar to the **c3** anomalies. Let's see how they are different. 

In [ ]:
(cola_nino34_c2 - cola_nino34_c3).mean('ensemble').plot(vmin=-0.2, vmax=0.2, cmap='bwr');

If one instead computes **c3** using the forecast error computed as the difference between the lead-time-dependent model climatology and the observed climatology over the period 1982-1998, then **c3** -> **c2**

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    fcst_er = cola_nino34_c0.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble']) - \
              obs_2_use['c0'].sel(init_date=clim_period, lead_time=0).groupby('init_date.month').mean('init_date')
    
    cola_nino34_c3_tmp = cola_nino34_c0.groupby('init_date.month') - fcst_er

    (cola_nino34_c2 - cola_nino34_c3_tmp).mean('ensemble').plot(vmin=-0.2, vmax=0.2, cmap='bwr');

Alternatively, if one computes **c0** using the "lead-time-dependent" observed climatology using **only** data within the period 1982-1998 and computes the errors as Delsole does, then **c3** -> **c2**, if **c2** anomalies are computed in a consistent way

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    
    init_dates = cola_nino34_c0_tmp.sel(init_date=clim_period).init_date.copy()
    lead_times = cola_nino34_c0_tmp.sel(init_date=clim_period).lead_time.copy()
    init_dates.values = np.arange(len(init_dates))
    mask = (init_dates + lead_times) <= init_dates[-1]
    
    clim = obs_2_use['full'].where(mask).groupby('init_date.month').mean('init_date')
    cola_nino34_c0_tmp = cola_nino34_full.groupby('init_date.month') - clim
    obs_c0_tmp = obs_2_use['full'].groupby('init_date.month') - clim

    fcst_er = (cola_nino34_c0_tmp.where(mask) - obs_c0_tmp.where(mask)).groupby('init_date.month').mean(['init_date','ensemble'])
    
    cola_nino34_c3_tmp = cola_nino34_c0_tmp.groupby('init_date.month') - fcst_er
    
    clim = cola_nino34_full.where(mask).groupby('init_date.month').mean(['init_date','ensemble'])
    cola_nino34_c2_tmp = cola_nino34_full.groupby('init_date.month') - clim

    (cola_nino34_c2_tmp - cola_nino34_c3_tmp).mean('ensemble').plot(vmin=-0.2, vmax=0.2, cmap='bwr');

What about performing the bias correction on the full field over the full period and then computing anomalies 

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    
    init_dates = cola_nino34_full.sel(init_date=clim_period).init_date.copy()
    lead_times = cola_nino34_full.sel(init_date=clim_period).lead_time.copy()
    init_dates.values = np.arange(len(init_dates))
    mask = (init_dates + lead_times) <= init_dates[-1]

    fcst_er = (cola_nino34_full.where(mask) - obs_2_use['full'].where(mask)).groupby('init_date.month').mean(['init_date','ensemble'])
    
    cola_nino34_tmp = cola_nino34_full.groupby('init_date.month') - fcst_er
   
    clim = cola_nino34_tmp.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    
    cola_nino34_c3_tmp = cola_nino34_full.groupby('init_date.month') - clim
    
    (cola_nino34_c3_tmp - cola_nino34_c3_tmp).mean('ensemble').plot(vmin=-0.2, vmax=0.2, cmap='bwr');

##### c1 full anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','2015')
    cola_nino34_full_use = cola_nino34_full.sel(init_date=clim_period).mean('ensemble')
    cola_nino34_c1full = cola_nino34_full.groupby('init_date').apply(cv_anomalize, clim_ts=cola_nino34_full_use).rename('c1full')
    
    cola_nino34_c1full.to_netcdf(dataloc + 'ENSO_predictability.cola_nino34_c1full.nc')
else:
    cola_nino34_c1full = xr.open_dataset(dataloc + 'ENSO_predictability.cola_nino34_c1full.nc')['c1full']

##### c1 ncv anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')

    clim = cola_nino34_full.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    cola_nino34_c1ncv = (cola_nino34_full.groupby('init_date.month') - clim).rename('c1ncv')
    
    cola_nino34_c1ncv.to_netcdf(dataloc + 'ENSO_predictability.cola_nino34_c1ncv.nc')
else:
    cola_nino34_c1ncv = xr.open_dataset(dataloc + 'ENSO_predictability.cola_nino34_c1ncv.nc')['c1ncv']

##### Combine into dataset

In [ ]:
cola_nino34 = cola_nino34_full.to_dataset()
cola_nino34['c0'] = cola_nino34_c0
cola_nino34['c1'] = cola_nino34_c1
cola_nino34['c2'] = cola_nino34_c2
cola_nino34['c3'] = cola_nino34_c3
cola_nino34['c1full'] = cola_nino34_c1full
cola_nino34['c1ncv'] = cola_nino34_c1ncv

### Raw GFDL-CM2p1-aer04 data

In [ ]:
if compute_data:
    filelist = glob.glob('/OSM/CBR/OA_DCFP/data2/model_output/NMME/phase1/GFDL-CM2p1-aer04/monthly/sst_mon_GFDL-CM2p1-aer04_198201_r*i1p1_198201-201811.nc')
    filelist.sort(key=natural_keys)

    aer04_sst_raw = xr.open_mfdataset(filelist, decode_times=False)['sst'] \
                     .rename({'X' : 'lon', 'Y' : 'lat', 'M' : 'ensemble', 
                              'L' : 'lead_time', 'S' : 'init_date'}).compute() - 273.15

    basedate = np.datetime64(aer04_sst_raw.init_date.units.split(' ')[2])
    aer04_sst_raw['init_date'] = np.array([doppyo.sugar.month_delta(basedate, int(shift)) for shift in aer04_sst_raw.init_date.values])

    aer04_sst_raw['lead_time'] = [int(lead_time - 0.5) for lead_time in aer04_sst_raw['lead_time']]
    aer04_sst_raw.lead_time.attrs['units'] = 'MS'

    aer04_sst_raw['ensemble'] = [int(ensemble) for ensemble in aer04_sst_raw['ensemble']]

##### Compute nino3.4

In [ ]:
if compute_data:
    aer04_nino34_full = doppyo.diagnostic.nino34(aer04_sst_raw).rename('full')
    
    aer04_nino34_full.to_netcdf(dataloc + 'ENSO_predictability.aer04_nino34_full.nc')
else:
    aer04_nino34_full = xr.open_dataset(dataloc + 'ENSO_predictability.aer04_nino34_full.nc')['full']

##### c0 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    clim = obs_2_use['full'].sel(init_date=clim_period, lead_time=0).groupby('init_date.month').mean('init_date').drop('lead_time')
    aer04_nino34_c0 = aer04_nino34_full.groupby('init_date').apply(lambda_anomalize, clim=clim).rename('c0')
    
    aer04_nino34_c0.to_netcdf(dataloc + 'ENSO_predictability.aer04_nino34_c0.nc')
else:
    aer04_nino34_c0 = xr.open_dataset(dataloc + 'ENSO_predictability.aer04_nino34_c0.nc')['c0']

##### c1 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')
    aer04_nino34_full_use = aer04_nino34_full.sel(init_date=clim_period).mean('ensemble')
    aer04_nino34_c1 = aer04_nino34_full.groupby('init_date').apply(cv_anomalize, clim_ts=aer04_nino34_full_use).rename('c1')
    
    aer04_nino34_c1.to_netcdf(dataloc + 'ENSO_predictability.aer04_nino34_c1.nc')
else:
    aer04_nino34_c1 = xr.open_dataset(dataloc + 'ENSO_predictability.aer04_nino34_c1.nc')['c1']

##### c2 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')

    clim = aer04_nino34_full.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    aer04_nino34_c2 = (aer04_nino34_full.groupby('init_date.month') - clim).rename('c2')
    
    aer04_nino34_c2.to_netcdf(dataloc + 'ENSO_predictability.aer04_nino34_c2.nc')
else:
    aer04_nino34_c2 = xr.open_dataset(dataloc + 'ENSO_predictability.aer04_nino34_c2.nc')['c2']

##### c3 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    
    init_dates = aer04_nino34_c0.sel(init_date=clim_period).init_date.copy()
    lead_times = aer04_nino34_c0.sel(init_date=clim_period).lead_time.copy()
    init_dates.values = np.arange(len(init_dates))
    mask = (init_dates + lead_times) <= init_dates[-1]

    fcst_er = (aer04_nino34_c0.where(mask) - obs_2_use['c0'].where(mask)).groupby('init_date.month').mean(['init_date','ensemble'])
    
    aer04_nino34_c3 = (aer04_nino34_c0.groupby('init_date.month') - fcst_er).rename('c3')

    aer04_nino34_c3.to_netcdf(dataloc + 'ENSO_predictability.aer04_nino34_c3.nc')
else:
    aer04_nino34_c3 = xr.open_dataset(dataloc + 'ENSO_predictability.aer04_nino34_c3.nc')['c3']

##### c1 full anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','2015')
    aer04_nino34_full_use = aer04_nino34_full.sel(init_date=clim_period).mean('ensemble')
    aer04_nino34_c1full = aer04_nino34_full.groupby('init_date').apply(cv_anomalize, clim_ts=aer04_nino34_full_use).rename('c1full')
    
    aer04_nino34_c1full.to_netcdf(dataloc + 'ENSO_predictability.aer04_nino34_c1full.nc')
else:
    aer04_nino34_c1full = xr.open_dataset(dataloc + 'ENSO_predictability.aer04_nino34_c1full.nc')['c1full']

##### c1 ncv anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')

    clim = aer04_nino34_full.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    aer04_nino34_c1ncv = (aer04_nino34_full.groupby('init_date.month') - clim).rename('c1ncv')
    
    aer04_nino34_c1ncv.to_netcdf(dataloc + 'ENSO_predictability.aer04_nino34_c1ncv.nc')
else:
    aer04_nino34_c1ncv = xr.open_dataset(dataloc + 'ENSO_predictability.aer04_nino34_c1ncv.nc')['c1ncv']

##### Combine into dataset

In [ ]:
aer04_nino34 = aer04_nino34_full.to_dataset()
aer04_nino34['c0'] = aer04_nino34_c0
aer04_nino34['c1'] = aer04_nino34_c1
aer04_nino34['c2'] = aer04_nino34_c2
aer04_nino34['c3'] = aer04_nino34_c3
aer04_nino34['c1full'] = aer04_nino34_c1full
aer04_nino34['c1ncv'] = aer04_nino34_c1ncv

### Raw GFDL-CM2p5-FLOR-A06 data

In [ ]:
if compute_data:
    filelist = glob.glob('/OSM/CBR/OA_DCFP/data2/model_output/NMME/phase1/GFDL-CM2p5-FLOR-A06/monthly/sst_mon_GFDL-CM2p5-FLOR-A06_198201_r*i1p1_198201-201811.nc')
    filelist.sort(key=natural_keys)

    florA_sst_raw = xr.open_mfdataset(filelist, decode_times=False)['sst'] \
                     .rename({'X' : 'lon', 'Y' : 'lat', 'M' : 'ensemble', 
                              'L' : 'lead_time', 'S' : 'init_date'}).compute()

    basedate = np.datetime64(florA_sst_raw.init_date.units.split(' ')[2])
    florA_sst_raw['init_date'] = np.array([doppyo.sugar.month_delta(basedate, int(shift)) for shift in florA_sst_raw.init_date.values])

    florA_sst_raw['lead_time'] = [int(lead_time - 0.5) for lead_time in florA_sst_raw['lead_time']]
    florA_sst_raw.lead_time.attrs['units'] = 'MS'

    florA_sst_raw['ensemble'] = [int(ensemble) for ensemble in florA_sst_raw['ensemble']]

##### Compute nino3.4

In [ ]:
if compute_data:
    florA_nino34_full = doppyo.diagnostic.nino34(florA_sst_raw).rename('full')
    
    florA_nino34_full.to_netcdf(dataloc + 'ENSO_predictability.florA_nino34_full.nc')
else:
    florA_nino34_full = xr.open_dataset(dataloc + 'ENSO_predictability.florA_nino34_full.nc')['full']

##### c0 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    clim = obs_2_use['full'].sel(init_date=clim_period, lead_time=0).groupby('init_date.month').mean('init_date').drop('lead_time')
    florA_nino34_c0 = florA_nino34_full.groupby('init_date').apply(lambda_anomalize, clim=clim).rename('c0')
    
    florA_nino34_c0.to_netcdf(dataloc + 'ENSO_predictability.florA_nino34_c0.nc')
else:
    florA_nino34_c0 = xr.open_dataset(dataloc + 'ENSO_predictability.florA_nino34_c0.nc')['c0']

##### c1 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')
    florA_nino34_full_use = florA_nino34_full.sel(init_date=clim_period).mean('ensemble')
    florA_nino34_c1 = florA_nino34_full.groupby('init_date').apply(cv_anomalize, clim_ts=florA_nino34_full_use).rename('c1')
    
    florA_nino34_c1.to_netcdf(dataloc + 'ENSO_predictability.florA_nino34_c1.nc')
else:
    florA_nino34_c1 = xr.open_dataset(dataloc + 'ENSO_predictability.florA_nino34_c1.nc')['c1']

##### c2 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')

    clim = florA_nino34_full.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    florA_nino34_c2 = (florA_nino34_full.groupby('init_date.month') - clim).rename('c2')
    
    florA_nino34_c2.to_netcdf(dataloc + 'ENSO_predictability.florA_nino34_c2.nc')
else:
    florA_nino34_c2 = xr.open_dataset(dataloc + 'ENSO_predictability.florA_nino34_c2.nc')['c2']

##### c3 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    
    init_dates = florA_nino34_c0.sel(init_date=clim_period).init_date.copy()
    lead_times = florA_nino34_c0.sel(init_date=clim_period).lead_time.copy()
    init_dates.values = np.arange(len(init_dates))
    mask = (init_dates + lead_times) <= init_dates[-1]

    fcst_er = (florA_nino34_c0.where(mask) - obs_2_use['c0'].where(mask)).groupby('init_date.month').mean(['init_date','ensemble'])
    
    florA_nino34_c3 = (florA_nino34_c0.groupby('init_date.month') - fcst_er).rename('c3')

    florA_nino34_c3.to_netcdf(dataloc + 'ENSO_predictability.florA_nino34_c3.nc')
else:
    florA_nino34_c3 = xr.open_dataset(dataloc + 'ENSO_predictability.florA_nino34_c3.nc')['c3']

##### c1 full anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','2015')
    florA_nino34_full_use = florA_nino34_full.sel(init_date=clim_period).mean('ensemble')
    florA_nino34_c1full = florA_nino34_full.groupby('init_date').apply(cv_anomalize, clim_ts=florA_nino34_full_use).rename('c1full')
    
    florA_nino34_c1full.to_netcdf(dataloc + 'ENSO_predictability.florA_nino34_c1full.nc')
else:
    florA_nino34_c1full = xr.open_dataset(dataloc + 'ENSO_predictability.florA_nino34_c1full.nc')['c1full']

##### c1 ncv anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')

    clim = florA_nino34_full.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    florA_nino34_c1ncv = (florA_nino34_full.groupby('init_date.month') - clim).rename('c1ncv')
    
    florA_nino34_c1ncv.to_netcdf(dataloc + 'ENSO_predictability.florA_nino34_c1ncv.nc')
else:
    florA_nino34_c1ncv = xr.open_dataset(dataloc + 'ENSO_predictability.florA_nino34_c1ncv.nc')['c1ncv']

##### Combine into dataset

In [ ]:
florA_nino34 = florA_nino34_full.to_dataset()
florA_nino34['c0'] = florA_nino34_c0
florA_nino34['c1'] = florA_nino34_c1
florA_nino34['c2'] = florA_nino34_c2
florA_nino34['c3'] = florA_nino34_c3
florA_nino34['c1full'] = florA_nino34_c1full
florA_nino34['c1ncv'] = florA_nino34_c1ncv

### Raw GFDL-CM2p5-FLOR-B01 data

In [ ]:
if compute_data:
    filelist = glob.glob('/OSM/CBR/OA_DCFP/data2/model_output/NMME/phase1/GFDL-CM2p5-FLOR-B01/monthly/sst_mon_GFDL-CM2p5-FLOR-B01_198201_r*i1p1_198201-201811.nc')
    filelist.sort(key=natural_keys)

    florB_sst_raw = xr.open_mfdataset(filelist, decode_times=False)['sst'] \
                     .rename({'X' : 'lon', 'Y' : 'lat', 'M' : 'ensemble', 
                              'L' : 'lead_time', 'S' : 'init_date'}).compute()

    basedate = np.datetime64(florB_sst_raw.init_date.units.split(' ')[2])
    florB_sst_raw['init_date'] = np.array([doppyo.sugar.month_delta(basedate, int(shift)) for shift in florB_sst_raw.init_date.values])

    florB_sst_raw['lead_time'] = [int(lead_time - 0.5) for lead_time in florB_sst_raw['lead_time']]
    florB_sst_raw.lead_time.attrs['units'] = 'MS'

    florB_sst_raw['ensemble'] = [int(ensemble) for ensemble in florB_sst_raw['ensemble']]

##### Compute nino3.4

In [ ]:
if compute_data:
    florB_nino34_full = doppyo.diagnostic.nino34(florB_sst_raw).rename('full')

    florB_nino34_full.to_netcdf(dataloc + 'ENSO_predictability.florB_nino34_full.nc')
else:
    florB_nino34_full = xr.open_dataset(dataloc + 'ENSO_predictability.florB_nino34_full.nc')['full']

##### c0 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    clim = obs_2_use['full'].sel(init_date=clim_period, lead_time=0).groupby('init_date.month').mean('init_date').drop('lead_time')
    florB_nino34_c0 = florB_nino34_full.groupby('init_date').apply(lambda_anomalize, clim=clim).rename('c0')
    
    florB_nino34_c0.to_netcdf(dataloc + 'ENSO_predictability.florB_nino34_c0.nc')
else:
    florB_nino34_c0 = xr.open_dataset(dataloc + 'ENSO_predictability.florB_nino34_c0.nc')['c0']

##### c1 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')
    florB_nino34_full_use = florB_nino34_full.sel(init_date=clim_period).mean('ensemble')
    florB_nino34_c1 = florB_nino34_full.groupby('init_date').apply(cv_anomalize, clim_ts=florB_nino34_full_use).rename('c1')
    
    florB_nino34_c1.to_netcdf(dataloc + 'ENSO_predictability.florB_nino34_c1.nc')
else:
    florB_nino34_c1 = xr.open_dataset(dataloc + 'ENSO_predictability.florB_nino34_c1.nc')['c1']

##### c2 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')

    clim = florB_nino34_full.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    florB_nino34_c2 = (florB_nino34_full.groupby('init_date.month') - clim).rename('c2')
    
    florB_nino34_c2.to_netcdf(dataloc + 'ENSO_predictability.florB_nino34_c2.nc')
else:
    florB_nino34_c2 = xr.open_dataset(dataloc + 'ENSO_predictability.florB_nino34_c2.nc')['c2']

##### c3 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    
    init_dates = florB_nino34_c0.sel(init_date=clim_period).init_date.copy()
    lead_times = florB_nino34_c0.sel(init_date=clim_period).lead_time.copy()
    init_dates.values = np.arange(len(init_dates))
    mask = (init_dates + lead_times) <= init_dates[-1]

    fcst_er = (florB_nino34_c0.where(mask) - obs_2_use['c0'].where(mask)).groupby('init_date.month').mean(['init_date','ensemble'])
    
    florB_nino34_c3 = (florB_nino34_c0.groupby('init_date.month') - fcst_er).rename('c3')

    florB_nino34_c3.to_netcdf(dataloc + 'ENSO_predictability.florB_nino34_c3.nc')
else:
    florB_nino34_c3 = xr.open_dataset(dataloc + 'ENSO_predictability.florB_nino34_c3.nc')['c3']

##### c1 full anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','2015')
    florB_nino34_full_use = florB_nino34_full.sel(init_date=clim_period).mean('ensemble')
    florB_nino34_c1full = florB_nino34_full.groupby('init_date').apply(cv_anomalize, clim_ts=florB_nino34_full_use).rename('c1full')
    
    florB_nino34_c1full.to_netcdf(dataloc + 'ENSO_predictability.florB_nino34_c1full.nc')
else:
    florB_nino34_c1full = xr.open_dataset(dataloc + 'ENSO_predictability.florB_nino34_c1full.nc')['c1full']

##### c1 ncv anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')

    clim = florB_nino34_full.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    florB_nino34_c1ncv = (florB_nino34_full.groupby('init_date.month') - clim).rename('c1ncv')
    
    florB_nino34_c1ncv.to_netcdf(dataloc + 'ENSO_predictability.florB_nino34_c1ncv.nc')
else:
    florB_nino34_c1ncv = xr.open_dataset(dataloc + 'ENSO_predictability.florB_nino34_c1ncv.nc')['c1ncv']

##### Combine into dataset

In [ ]:
florB_nino34 = florB_nino34_full.to_dataset()
florB_nino34['c0'] = florB_nino34_c0
florB_nino34['c1'] = florB_nino34_c1
florB_nino34['c2'] = florB_nino34_c2
florB_nino34['c3'] = florB_nino34_c3
florB_nino34['c1full'] = florB_nino34_c1full
florB_nino34['c1ncv'] = florB_nino34_c1ncv

### Raw CMC CanCM3 data

In [ ]:
if compute_data:
    folder = '/OSM/CBR/OA_DCFP/data2/model_output/NMME/phase1/CMC_CanCM3/monthly/'
    hcstfiles = 'sst_mon_CanCM3_198101*.nc'
    fcstfiles = 'sst_mon_CanCM3_201101*.nc'

    hcstlist = glob.glob(folder + hcstfiles)
    hcstlist.sort(key=natural_keys)
    fcstlist = glob.glob(folder + fcstfiles)
    fcstlist.sort(key=natural_keys)

    cm3_sst_raw = xr.concat([xr.open_mfdataset(hcstlist, decode_times=False)['sst'] - 273.15,
                             xr.open_mfdataset(fcstlist, decode_times=False)['sst'] - 273.15], dim='S') \
                    .rename({'X' : 'lon', 'Y' : 'lat', 'M' : 'ensemble', 
                             'L' : 'lead_time', 'S' : 'init_date'}).compute()

    basedate = np.datetime64(cm3_sst_raw.init_date.units.split(' ')[2])
    cm3_sst_raw['init_date'] = np.array([doppyo.sugar.month_delta(basedate, int(shift)) for shift in cm3_sst_raw.init_date.values])

    cm3_sst_raw['lead_time'] = [int(lead_time - 0.5) for lead_time in cm3_sst_raw['lead_time']]
    cm3_sst_raw.lead_time.attrs['units'] = 'MS'

    cm3_sst_raw['ensemble'] = [int(ensemble) for ensemble in cm3_sst_raw['ensemble']]

##### Compute nino3.4

In [ ]:
if compute_data:
    cm3_nino34_full = doppyo.diagnostic.nino34(cm3_sst_raw).rename('full')

    cm3_nino34_full.to_netcdf(dataloc + 'ENSO_predictability.cm3_nino34_full.nc')
else:
    cm3_nino34_full = xr.open_dataset(dataloc + 'ENSO_predictability.cm3_nino34_full.nc')['full']

##### c0 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    clim = obs_2_use['full'].sel(init_date=clim_period, lead_time=0).groupby('init_date.month').mean('init_date').drop('lead_time')
    cm3_nino34_c0 = cm3_nino34_full.groupby('init_date').apply(lambda_anomalize, clim=clim).rename('c0')
    
    cm3_nino34_c0.to_netcdf(dataloc + 'ENSO_predictability.cm3_nino34_c0.nc')
else:
    cm3_nino34_c0 = xr.open_dataset(dataloc + 'ENSO_predictability.cm3_nino34_c0.nc')['c0']

##### c1 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')
    cm3_nino34_full_use = cm3_nino34_full.sel(init_date=clim_period).mean('ensemble')
    cm3_nino34_c1 = cm3_nino34_full.groupby('init_date').apply(cv_anomalize, clim_ts=cm3_nino34_full_use).rename('c1')
    
    cm3_nino34_c1.to_netcdf(dataloc + 'ENSO_predictability.cm3_nino34_c1.nc')
else:
    cm3_nino34_c1 = xr.open_dataset(dataloc + 'ENSO_predictability.cm3_nino34_c1.nc')['c1']

##### c2 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')

    clim = cm3_nino34_full.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    cm3_nino34_c2 = (cm3_nino34_full.groupby('init_date.month') - clim).rename('c2')
    
    cm3_nino34_c2.to_netcdf(dataloc + 'ENSO_predictability.cm3_nino34_c2.nc')
else:
    cm3_nino34_c2 = xr.open_dataset(dataloc + 'ENSO_predictability.cm3_nino34_c2.nc')['c2']

##### c3 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    
    init_dates = cm3_nino34_c0.sel(init_date=clim_period).init_date.copy()
    lead_times = cm3_nino34_c0.sel(init_date=clim_period).lead_time.copy()
    init_dates.values = np.arange(len(init_dates))
    mask = (init_dates + lead_times) <= init_dates[-1]

    fcst_er = (cm3_nino34_c0.where(mask) - obs_2_use['c0'].where(mask)).groupby('init_date.month').mean(['init_date','ensemble'])
    
    cm3_nino34_c3 = (cm3_nino34_c0.groupby('init_date.month') - fcst_er).rename('c3')

    cm3_nino34_c3.to_netcdf(dataloc + 'ENSO_predictability.cm3_nino34_c3.nc')
else:
    cm3_nino34_c3 = xr.open_dataset(dataloc + 'ENSO_predictability.cm3_nino34_c3.nc')['c3']

##### c1 full anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','2015')
    cm3_nino34_full_use = cm3_nino34_full.sel(init_date=clim_period).mean('ensemble')
    cm3_nino34_c1full = cm3_nino34_full.groupby('init_date').apply(cv_anomalize, clim_ts=cm3_nino34_full_use).rename('c1full')
    
    cm3_nino34_c1full.to_netcdf(dataloc + 'ENSO_predictability.cm3_nino34_c1full.nc')
else:
    cm3_nino34_c1full = xr.open_dataset(dataloc + 'ENSO_predictability.cm3_nino34_c1full.nc')['c1full']

##### c1 ncv anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')

    clim = cm3_nino34_full.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    cm3_nino34_c1ncv = (cm3_nino34_full.groupby('init_date.month') - clim).rename('c1ncv')
    
    cm3_nino34_c1ncv.to_netcdf(dataloc + 'ENSO_predictability.cm3_nino34_c1ncv.nc')
else:
    cm3_nino34_c1ncv = xr.open_dataset(dataloc + 'ENSO_predictability.cm3_nino34_c1ncv.nc')['c1ncv']

##### Combine into dataset

In [ ]:
cm3_nino34 = cm3_nino34_full.to_dataset()
cm3_nino34['c0'] = cm3_nino34_c0
cm3_nino34['c1'] = cm3_nino34_c1
cm3_nino34['c2'] = cm3_nino34_c2
cm3_nino34['c3'] = cm3_nino34_c3
cm3_nino34['c1full'] = cm3_nino34_c1full
cm3_nino34['c1ncv'] = cm3_nino34_c1ncv

### Raw CMC CanCM4 data

In [ ]:
if compute_data:
    folder = '/OSM/CBR/OA_DCFP/data2/model_output/NMME/phase1/CMC_CanCM4/monthly/'
    hcstfiles = 'sst_mon_CanCM4_198101*.nc'
    fcstfiles = 'sst_mon_CanCM4_201101*.nc'

    hcstlist = glob.glob(folder + hcstfiles)
    hcstlist.sort(key=natural_keys)
    fcstlist = glob.glob(folder + fcstfiles)
    fcstlist.sort(key=natural_keys)

    cm4_sst_raw = xr.concat([xr.open_mfdataset(hcstlist, decode_times=False)['sst'] - 273.15,
                             xr.open_mfdataset(fcstlist, decode_times=False)['sst'] - 273.15], dim='S') \
                    .rename({'X' : 'lon', 'Y' : 'lat', 'M' : 'ensemble', 
                             'L' : 'lead_time', 'S' : 'init_date'}).compute()

    basedate = np.datetime64(cm4_sst_raw.init_date.units.split(' ')[2])
    cm4_sst_raw['init_date'] = np.array([doppyo.sugar.month_delta(basedate, int(shift)) for shift in cm4_sst_raw.init_date.values])

    cm4_sst_raw['lead_time'] = [int(lead_time - 0.5) for lead_time in cm4_sst_raw['lead_time']]
    cm4_sst_raw.lead_time.attrs['units'] = 'MS'

    cm4_sst_raw['ensemble'] = [int(ensemble) for ensemble in cm4_sst_raw['ensemble']]

##### Compute nino3.4

In [ ]:
if compute_data:
    cm4_nino34_full = doppyo.diagnostic.nino34(cm4_sst_raw).rename('full')

    cm4_nino34_full.to_netcdf(dataloc + 'ENSO_predictability.cm4_nino34_full.nc')
else:
    cm4_nino34_full = xr.open_dataset(dataloc + 'ENSO_predictability.cm4_nino34_full.nc')['full']

##### c0 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    clim = obs_2_use['full'].sel(init_date=clim_period, lead_time=0).groupby('init_date.month').mean('init_date').drop('lead_time')
    cm4_nino34_c0 = cm4_nino34_full.groupby('init_date').apply(lambda_anomalize, clim=clim).rename('c0')
    
    cm4_nino34_c0.to_netcdf(dataloc + 'ENSO_predictability.cm4_nino34_c0.nc')
else:
    cm4_nino34_c0 = xr.open_dataset(dataloc + 'ENSO_predictability.cm4_nino34_c0.nc')['c0']

##### c1 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')
    cm4_nino34_full_use = cm4_nino34_full.sel(init_date=clim_period).mean('ensemble')
    cm4_nino34_c1 = cm4_nino34_full.groupby('init_date').apply(cv_anomalize, clim_ts=cm4_nino34_full_use).rename('c1')
    
    cm4_nino34_c1.to_netcdf(dataloc + 'ENSO_predictability.cm4_nino34_c1.nc')
else:
    cm4_nino34_c1 = xr.open_dataset(dataloc + 'ENSO_predictability.cm4_nino34_c1.nc')['c1']

##### c2 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')

    clim = cm4_nino34_full.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    cm4_nino34_c2 = (cm4_nino34_full.groupby('init_date.month') - clim).rename('c2')
    
    cm4_nino34_c2.to_netcdf(dataloc + 'ENSO_predictability.cm4_nino34_c2.nc')
else:
    cm4_nino34_c2 = xr.open_dataset(dataloc + 'ENSO_predictability.cm4_nino34_c2.nc')['c2']

##### c3 anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','1998')
    
    init_dates = cm4_nino34_c0.sel(init_date=clim_period).init_date.copy()
    lead_times = cm4_nino34_c0.sel(init_date=clim_period).lead_time.copy()
    init_dates.values = np.arange(len(init_dates))
    mask = (init_dates + lead_times) <= init_dates[-1]

    fcst_er = (cm4_nino34_c0.where(mask) - obs_2_use['c0'].where(mask)).groupby('init_date.month').mean(['init_date','ensemble'])
    
    cm4_nino34_c3 = (cm4_nino34_c0.groupby('init_date.month') - fcst_er).rename('c3')

    cm4_nino34_c3.to_netcdf(dataloc + 'ENSO_predictability.cm4_nino34_c3.nc')
else:
    cm4_nino34_c3 = xr.open_dataset(dataloc + 'ENSO_predictability.cm4_nino34_c3.nc')['c3']

##### c1 full anomalies

In [ ]:
if compute_data:
    clim_period = slice('1982','2015')
    cm4_nino34_full_use = cm4_nino34_full.sel(init_date=clim_period).mean('ensemble')
    cm4_nino34_c1full = cm4_nino34_full.groupby('init_date').apply(cv_anomalize, clim_ts=cm4_nino34_full_use).rename('c1full')
    
    cm4_nino34_c1full.to_netcdf(dataloc + 'ENSO_predictability.cm4_nino34_c1full.nc')
else:
    cm4_nino34_c1full = xr.open_dataset(dataloc + 'ENSO_predictability.cm4_nino34_c1full.nc')['c1full']

##### c1 ncv anomalies

In [ ]:
if compute_data:
    clim_period = slice('1999','2015')

    clim = cm4_nino34_full.sel(init_date=clim_period).groupby('init_date.month').mean(['init_date','ensemble'])
    cm4_nino34_c1ncv = (cm4_nino34_full.groupby('init_date.month') - clim).rename('c1ncv')
    
    cm4_nino34_c1ncv.to_netcdf(dataloc + 'ENSO_predictability.cm4_nino34_c1ncv.nc')
else:
    cm4_nino34_c1ncv = xr.open_dataset(dataloc + 'ENSO_predictability.cm4_nino34_c1ncv.nc')['c1ncv']

##### Combine into dataset

In [ ]:
cm4_nino34 = cm4_nino34_full.to_dataset()
cm4_nino34['c0'] = cm4_nino34_c0
cm4_nino34['c1'] = cm4_nino34_c1
cm4_nino34['c2'] = cm4_nino34_c2
cm4_nino34['c3'] = cm4_nino34_c3
cm4_nino34['c1full'] = cm4_nino34_c1full
cm4_nino34['c1ncv'] = cm4_nino34_c1ncv

# Multi-model mean

In [ ]:
mmm_nino34 = xr.concat([cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34], dim='ensemble').mean('ensemble')

# Ordinary linear regression model
For each of c0, c1, c2 and c3, the linear regression model is compute in a way consistent with how the observed anomalies are computed:

**c0** : the regression model is trained on observed c0 anomalies over the period **1982-01 -> 1998-12** and apply over **1999-01 -> 2015-12**

**c1** : the regression model is trained on observed c1 anomalies over the period **1999-01 -> 2015-12** using a cross-validation approach and then applied over the same period

**c2** : the regression model is trained on observed c2 anomalies (same as c1) over the period **1982-01 -> 1998-12** and apply over **1999-01 -> 2015-12**

**c1full** : the regression model is trained on observed c1 anomalies over the period **1982-01 -> 2015-12** using a cross-validation approach and then applied over the same period

##### c0 regression model

In [ ]:
train_period = slice('1982','1998')
predict_period = slice('1999','2015')
regr_nino34_c0 = Delsole_regression(obs_2_use['c0'].sel(init_date=predict_period, lead_time=0),
                                    obs_2_use['c0'].sel(init_date=train_period)).rename('c0')

##### c1 regression model

In [ ]:
train_period = slice('1999','2015')
predict_period = slice('1999','2015')
regr_nino34_c1 = obs_2_use['c1'].sel(init_date=predict_period).sel(lead_time=0,drop=True).groupby('init_date') \
                                 .apply(Delsole_regression, da_train=obs_2_use['c1'].sel(init_date=train_period)).rename('c1')

##### c2 regression model

In [ ]:
regr_nino34_c2 = regr_nino34_c0.rename('c2')

##### c3 regression model

In [ ]:
regr_nino34_c3 = regr_nino34_c0.rename('c3')

##### c1 full regression model

In [ ]:
train_period = slice('1982','2015')
predict_period = slice('1982','2015')
regr_nino34_c1full = obs_2_use['c1full'].sel(init_date=predict_period).sel(lead_time=0,drop=True).groupby('init_date') \
                                 .apply(Delsole_regression, da_train=obs_2_use['c1full'].sel(init_date=train_period)).rename('c1full')

##### Combine into dataset

In [ ]:
regr_nino34 = regr_nino34_c0.to_dataset()
regr_nino34['c1'] = regr_nino34_c1
regr_nino34['c2'] = regr_nino34_c2
regr_nino34['c3'] = regr_nino34_c3
regr_nino34['c1full'] = regr_nino34_c1full

# Get ENSO events

In [ ]:
where_elnino = obs_2_use.apply(where_elninos).drop('full')
where_lanina = obs_2_use.apply(where_laninas).drop('full')

where_elnino_cola = cola_nino34.mean('ensemble').apply(where_elninos).drop('full')
where_lanina_cola = cola_nino34.mean('ensemble').apply(where_laninas).drop('full')

where_elnino_aer04 = aer04_nino34.mean('ensemble').apply(where_elninos).drop('full')
where_lanina_aer04 = aer04_nino34.mean('ensemble').apply(where_laninas).drop('full')

where_elnino_florA = florA_nino34.mean('ensemble').apply(where_elninos).drop('full')
where_lanina_florA = florA_nino34.mean('ensemble').apply(where_laninas).drop('full')

where_elnino_florB = florB_nino34.mean('ensemble').apply(where_elninos).drop('full')
where_lanina_florB = florB_nino34.mean('ensemble').apply(where_laninas).drop('full')

where_elnino_cm3 = cm3_nino34.mean('ensemble').apply(where_elninos).drop('full')
where_lanina_cm3 = cm3_nino34.mean('ensemble').apply(where_laninas).drop('full')

where_elnino_cm4 = cm4_nino34.mean('ensemble').apply(where_elninos).drop('full')
where_lanina_cm4 = cm4_nino34.mean('ensemble').apply(where_laninas).drop('full')

where_elnino_regr = regr_nino34.apply(where_elninos)
where_lanina_regr = regr_nino34.apply(where_laninas)

In [ ]:
linewidth = 2
fontsize = 12
grey = [0.9,0.9,0.9]

matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': fontsize})

anom='c2'
figure = plt.figure(figsize=(12,6))
fcst_dates = pd.date_range(start='2006-01', end='2013-01', freq='MS')[::-1]
    
for idx, date in enumerate(fcst_dates):
    if idx == 0:
        plot_fcst(doppyo.utils.leadtime_to_datetime(mmm_nino34[anom].sel(init_date=date)).time,
                  doppyo.utils.leadtime_to_datetime(mmm_nino34[anom].sel(init_date=date)), cmap='winter', 
                  **{'linewidth':linewidth,'label':'NMME multi-model ensemble average'})
    else:
        plot_fcst(doppyo.utils.leadtime_to_datetime(mmm_nino34[anom].sel(init_date=date)).time,
                  doppyo.utils.leadtime_to_datetime(mmm_nino34[anom].sel(init_date=date)), cmap='winter', 
                  **{'linewidth':linewidth,'label':'_nolegend_'})
    
for idx, date in enumerate(fcst_dates):
    if idx == 0:
        plot_fcst(doppyo.utils.leadtime_to_datetime(regr_nino34[anom].sel(init_date=date)).time,
                  doppyo.utils.leadtime_to_datetime(regr_nino34[anom].sel(init_date=date)), cmap='autumn', 
                  **{'linewidth':linewidth,'label':'Linear regression'})
    else:
        plot_fcst(doppyo.utils.leadtime_to_datetime(regr_nino34[anom].sel(init_date=date)).time,
                  doppyo.utils.leadtime_to_datetime(regr_nino34[anom].sel(init_date=date)), cmap='autumn', 
                  **{'linewidth':linewidth,'label':'_nolegend_'})

for idx, date in enumerate(fcst_dates):
    if idx == 0:
        plt.plot(doppyo.utils.leadtime_to_datetime(cola_nino34[anom].sel(init_date=date)).time.values,
                 doppyo.utils.leadtime_to_datetime(cola_nino34[anom].sel(init_date=date)).mean('ensemble'),
                 color=grey, zorder=1, label='NMME individual model ensemble average', linewidth=linewidth)
    else:
        plt.plot(doppyo.utils.leadtime_to_datetime(cola_nino34[anom].sel(init_date=date)).time.values,
                 doppyo.utils.leadtime_to_datetime(cola_nino34[anom].sel(init_date=date)).mean('ensemble'), 
                 color=grey, zorder=1, label='_nolegend_', linewidth=linewidth)
for date in fcst_dates:
    plt.plot(doppyo.utils.leadtime_to_datetime(aer04_nino34[anom].sel(init_date=date)).time.values,
                 doppyo.utils.leadtime_to_datetime(aer04_nino34[anom].sel(init_date=date)).mean('ensemble'), 
                 color=grey, zorder=1, label='_nolegend_', linewidth=linewidth)
for date in fcst_dates:
    plt.plot(doppyo.utils.leadtime_to_datetime(florA_nino34[anom].sel(init_date=date)).time.values,
                 doppyo.utils.leadtime_to_datetime(florA_nino34[anom].sel(init_date=date)).mean('ensemble'), 
                 color=grey, zorder=1, label='_nolegend_', linewidth=linewidth)
for date in fcst_dates:
    plt.plot(doppyo.utils.leadtime_to_datetime(florB_nino34[anom].sel(init_date=date)).time.values,
                 doppyo.utils.leadtime_to_datetime(florB_nino34[anom].sel(init_date=date)).mean('ensemble'), 
                 color=grey, zorder=1, label='_nolegend_', linewidth=linewidth)
for date in fcst_dates:
    plt.plot(doppyo.utils.leadtime_to_datetime(cm3_nino34[anom].sel(init_date=date)).time.values,
                 doppyo.utils.leadtime_to_datetime(cm3_nino34[anom].sel(init_date=date)).mean('ensemble'), 
                 color=grey, zorder=1, label='_nolegend_', linewidth=linewidth)
for date in fcst_dates:
    plt.plot(doppyo.utils.leadtime_to_datetime(cm4_nino34[anom].sel(init_date=date)).time.values,
                 doppyo.utils.leadtime_to_datetime(cm4_nino34[anom].sel(init_date=date)).mean('ensemble'), 
                 color=grey, zorder=1, label='_nolegend_', linewidth=linewidth)

# for idx, date in enumerate(fcst_dates):
#     plt.plot(doppyo.utils.leadtime_to_datetime(fcst_use.sel(init_date=date)).time.values,
#              doppyo.utils.leadtime_to_datetime(fcst_use.sel(init_date=date)), color=(0.95,0.95,0.95), zorder=0, label='_nolegend_')

obs_2_use[anom].sel(lead_time=0).plot(color='k', linestyle='-', linewidth=linewidth+1, label='Observations')

plt.xlim('2007','2013')
plt.ylim(-3.8,3.8);
plt.xlabel('time');
plt.ylabel('NINO-3.4');
plt.title('')

# Make the legend -----
span = pd.date_range('2007-02-01','2007-05-01', freq='15D')
plt.plot(span, 3.5*np.ones_like(span.astype(int)), color=grey, zorder=1, linewidth=linewidth)
plt.text('2007-05-25', 3.4, 'NMME individual model ensemble average', fontsize=fontsize-1)
plot_fcst(span, 3.15*np.ones_like(span.astype(int)), cmap='winter', **{'linewidth':linewidth})
plt.text('2007-05-25', 3.05, 'NMME multi-model ensemble average', fontsize=fontsize-1)
plot_fcst(span, 2.8*np.ones_like(span.astype(int)), cmap='autumn', **{'linewidth':linewidth})
plt.text('2007-05-25', 2.7, 'Linear regression', fontsize=fontsize-1)
plt.plot(span, 2.45*np.ones_like(span.astype(int)), color='k', linestyle='-', linewidth=linewidth+1)
plt.text('2007-05-25', 2.35, 'Observations', fontsize=fontsize-1)

plt.tight_layout();

# How do the different anomalies compare?

In [ ]:
lead=11
plt.figure(figsize=(20,8))

cola_nino34['c0'].sel(lead_time=lead).mean('ensemble').plot(linestyle='-', color='#1f77b4', label='_nolegend_')
cola_nino34['c1'].sel(lead_time=lead).mean('ensemble').plot(linestyle='--', color='#1f77b4', label='_nolegend_')
cola_nino34['c2'].sel(lead_time=lead).mean('ensemble').plot(linestyle='-.', color='#1f77b4', label='_nolegend_')

# aer04_nino34['c0'].sel(lead_time=lead).mean('ensemble').plot(linestyle='-', color='#ff7f0e', label='_nolegend_')
# aer04_nino34['c1'].sel(lead_time=lead).mean('ensemble').plot(linestyle='--', color='#ff7f0e', label='_nolegend_')
# aer04_nino34['c2'].sel(lead_time=lead).mean('ensemble').plot(linestyle='-.', color='#ff7f0e', label='_nolegend_')

# florA_nino34['c0'].sel(lead_time=lead).mean('ensemble').plot(linestyle='-', color='#2ca02c', label='_nolegend_')
# florA_nino34['c1'].sel(lead_time=lead).mean('ensemble').plot(linestyle='--', color='#2ca02c', label='_nolegend_')
# florA_nino34['c2'].sel(lead_time=lead).mean('ensemble').plot(linestyle='-.', color='#2ca02c', label='_nolegend_')

# florB_nino34['c0'].sel(lead_time=lead).mean('ensemble').plot(linestyle='-', color='#d62728', label='_nolegend_')
# florB_nino34['c1'].sel(lead_time=lead).mean('ensemble').plot(linestyle='--', color='#d62728', label='_nolegend_')
# florB_nino34['c2'].sel(lead_time=lead).mean('ensemble').plot(linestyle='-.', color='#d62728', label='_nolegend_')

# cm3_nino34['c0'].sel(lead_time=lead).mean('ensemble').plot(linestyle='-', color='#9467bd', label='_nolegend_')
# cm3_nino34['c1'].sel(lead_time=lead).mean('ensemble').plot(linestyle='--', color='#9467bd', label='_nolegend_')
# cm3_nino34['c2'].sel(lead_time=lead).mean('ensemble').plot(linestyle='-.', color='#9467bd', label='_nolegend_')

# cm4_nino34['c0'].sel(lead_time=lead).mean('ensemble').plot(linestyle='-', color='#8c564b', label='_nolegend_')
# cm4_nino34['c1'].sel(lead_time=lead).mean('ensemble').plot(linestyle='--', color='#8c564b', label='_nolegend_')
# cm4_nino34['c2'].sel(lead_time=lead).mean('ensemble').plot(linestyle='-.', color='#8c564b', label='_nolegend_')

obs_2_use['c0'].sel(lead_time=lead).plot(linestyle='-', color='k', label='c0')
obs_2_use['c1'].sel(lead_time=lead).plot(linestyle='--', color='k', label='c1')
obs_2_use['c2'].sel(lead_time=lead).plot(linestyle='-.', color='k', label='c2')

plt.fill_between(where_elnino['c0'].init_date.values, -5, (100*where_elnino['c0'].sel(lead_time=lead)-10), color='red', alpha=0.1, edgecolor=None)
plt.fill_between(where_lanina['c0'].init_date.values, -5, (100*where_lanina['c0'].sel(lead_time=lead)-10), color='blue', alpha=0.1, edgecolor=None)
# plt.fill_between(where_elnino['c1'].init_date.values, -5, (100*where_elnino['c1'].sel(lead_time=lead)-10), color='magenta', alpha=0.1, edgecolor=None)
# plt.fill_between(where_lanina['c1'].init_date.values, -5, (100*where_lanina['c1'].sel(lead_time=lead)-10), color='purple', alpha=0.1, edgecolor=None)

plt.legend()
plt.ylim(-4,4);

# First, let's get an appreciation of the NMME skill by looking at contingency tables for El Nino and La Nina

In [ ]:
which_state = where_lanina.where(where_lanina == 0, other=-1) + where_elnino.where(where_elnino == 0, other=1)

which_state_cola = where_lanina_cola.where(where_lanina_cola == 0, other=-1) + where_elnino_cola.where(where_elnino_cola == 0, other=1)
which_state_aer04 = where_lanina_aer04.where(where_lanina_aer04 == 0, other=-1) + where_elnino_aer04.where(where_elnino_aer04 == 0, other=1)
which_state_florA = where_lanina_florA.where(where_lanina_florA == 0, other=-1) + where_elnino_florA.where(where_elnino_florA == 0, other=1)
which_state_florB = where_lanina_florB.where(where_lanina_florB == 0, other=-1) + where_elnino_florB.where(where_elnino_florB == 0, other=1)
which_state_cm3 = where_lanina_cm3.where(where_lanina_cm3 == 0, other=-1) + where_elnino_cm3.where(where_elnino_cm3 == 0, other=1)
which_state_cm4 = where_lanina_cm4.where(where_lanina_cm4 == 0, other=-1) + where_elnino_cm4.where(where_elnino_cm4 == 0, other=1)

which_state_regr = where_lanina_regr.where(where_lanina_regr == 0, other=-1) + where_elnino_regr.where(where_elnino_regr == 0, other=1)

In [ ]:
anoms = ['c0','c1','c2']
time_period = slice('1999','2015')

fig = plt.figure(figsize=(5*len(anoms),9))
axes = fig.subplots(3, ncols=len(anoms), sharex=True, sharey=True)
Gerrity = []
for idx, anom in enumerate(anoms):
    
    da_cmp = which_state_cm4[anom].sel(init_date=time_period)
    da_ref = which_state[anom].sel(init_date=time_period)
    edge = [-np.inf, -0.5, 0.5, np.inf]
    contingency = doppyo.skill.contingency(da_cmp, da_ref, edge, edge, over_dims='init_date')
    
    Gerrity.append(doppyo.skill.Gerrity_score(contingency))
    
    for idy in range(3):
        ax = axes[idy, idx]
    
        if idy == 0:
            total = contingency.sel(reference_category=1).sum('comparison_category')
            (contingency.sel(reference_category=1, comparison_category=1) / total).plot(ax=ax, color='b', linestyle='-', label='La Nina - La Nina')
            (contingency.sel(reference_category=1, comparison_category=2) / total).plot(ax=ax, color='b', linestyle='--', label='La Nina - Neutral')
            (contingency.sel(reference_category=1, comparison_category=3) / total).plot(ax=ax, color='b', linestyle=':', label='La Nina - El Nino')

        elif idy == 1:
            total = contingency.sel(reference_category=2).sum('comparison_category')
            (contingency.sel(reference_category=2, comparison_category=1) / total).plot(ax=ax, color='k', linestyle='--', label='Neutral - La Nina')
            (contingency.sel(reference_category=2, comparison_category=2) / total).plot(ax=ax, color='k', linestyle='-', label='Neutral - Neutral')
            (contingency.sel(reference_category=2, comparison_category=3) / total).plot(ax=ax, color='k', linestyle=':', label='Neutral - El Nino')

        elif idy == 2:
            total = contingency.sel(reference_category=3).sum('comparison_category')
            (contingency.sel(reference_category=3, comparison_category=1) / total).plot(ax=ax, color='r', linestyle='--', label='El Nino - La Nina')
            (contingency.sel(reference_category=3, comparison_category=2) / total).plot(ax=ax, color='r', linestyle=':', label='El Nino - Neutral')
            (contingency.sel(reference_category=3, comparison_category=3) / total).plot(ax=ax, color='r', linestyle='-', label='El Nino - El Nino')
    
        ax.set_title('')
        if idx != 0:
            ax.set_ylabel('')
        if idy != 2:
            ax.set_xlabel('')
        if idx == 0:
            ax.legend()
    ax.set_ylim(-0.1,1.1)

fig.tight_layout()

# Maybe this isn't the most intuitive way to present absolute skill. What about the Gerrity score?

In [ ]:
models = [which_state_cola, which_state_aer04, which_state_florA, which_state_florB, which_state_cm3, which_state_cm4, which_state_regr]

anoms = ['c0','c1','c2']
fig = plt.figure(figsize=(5*len(anoms),3*1))
axes = fig.subplots(nrows=1, ncols=len(anoms), sharex=True)

for idy, anom in enumerate(anoms):
    for idx, model in enumerate(models):
        ax = axes[idy]
        da_cmp = model[anom].sel(init_date=time_period)
        da_ref = which_state[anom].sel(init_date=time_period)
        edge = [-np.inf, -0.5, 0.5, np.inf]
        contingency = doppyo.skill.contingency(da_cmp, da_ref, edge, edge, over_dims='init_date')

        doppyo.skill.Gerrity_score(contingency).plot(ax=ax)
    
        ax.set_ylim(0,1);
fig.tight_layout()

# Still not entirely intuitive. What about mean squared skill score (MSSS) and anomaly cross correlation (ACC)?
Focus on **c3**

In [ ]:
linewidth = 2
fontsize = 12

matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': fontsize})

anom = 'c2'
models = [cola_nino34.mean('ensemble'), aer04_nino34.mean('ensemble'), florA_nino34.mean('ensemble'), 
          florB_nino34.mean('ensemble'), cm3_nino34.mean('ensemble'), cm4_nino34.mean('ensemble'), regr_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4','linear regression']

fig = plt.figure(figsize=(12,4))
axes = fig.subplots(nrows=1, ncols=2, sharex=True)
for idx, model in enumerate(models):
    ax0 = axes[0]
    MSE_f = doppyo.skill.mean_squared_error(model[anom], obs_2_use[anom], over_dims='init_date')
    MSE_o = doppyo.skill.mean_squared_error(0*obs_2_use[anom], obs_2_use[anom], over_dims='init_date')
    ax0.plot([-1,12],[0,0],'-',color=[0.85,0.85,0.85], zorder=0)
    (1 - MSE_f / MSE_o).plot(ax=ax0, label=model_name[idx], linewidth=linewidth, color=colors[idx])
    legend = ax0.legend(ncol=2, fontsize=fontsize-1)
    legend.get_frame().set_facecolor('w')
    legend.get_frame().set_edgecolor('w')
    ax0.set_xlim(0,11)
    ax0.set_ylim(-1,1)
    ax0.set_ylabel('MSSS')
    ax0.set_xlabel('lead [months]')

    ax1 = axes[1]
    ACC = doppyo.skill.Pearson_corrcoeff(model[anom], obs_2_use[anom], over_dims='init_date')
    ax1.plot([-1,12],[0,0],'-',color=[0.85,0.85,0.85], zorder=0)
    ACC.plot(ax=ax1, label=model_name[idx], linewidth=linewidth, color=colors[idx])
    ax1.set_ylim(-1,1)
    ax1.set_ylabel('ACC')
    ax1.set_xlabel('lead [months]')
    
fig.tight_layout()

# What is the (sign test) skill relative to linear regression for each of the different anomaly types?

In [ ]:
lead = 2
anoms = ['c0','c1','c2','c3']
masks = [1 + 0*where_elnino]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            sign, conf = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                                model[anom].mean('ensemble').where(mask[anom]), 
                                                obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
                
            if idz == 0:
                ax.fill_between(conf.init_date.values,
                                -1 * conf.sel(lead_time=lead), 
                                conf.sel(lead_time=lead), color='gray', alpha=0.2, label='_nolegend_')
            ax.plot(sign.init_date, sign.sel(lead_time=lead), label=model_name[idz])
            
        if (idx == 0) & (idy == 0):
            ax.legend()

fig.tight_layout()

### Plot the area under the curves as a function of lead time
Note that when computing the areas, it is important to only include points where the sign_test had the opportunity to change - ie. when supplying masks to the sign_test, the ssame amsks put also be applied before the integral is calculated

In [ ]:
anoms = ['c0','c1','c2','c3']
masks = [1 + 0*where_elnino]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            sign_test = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                               model[anom].mean('ensemble').where(mask[anom]), 
                                               obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
            
            sign_masked = sign_test[0].where(mask[anom], drop=True)
            conf_masked = sign_test[1].where(mask[anom], drop=True)
            sign_area = doppyo.utils.integrate(sign_masked, over_dim='init_date', 
                                               x=(1+0*sign_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            conf_area = doppyo.utils.integrate(conf_masked, over_dim='init_date', 
                                               x=(1+0*conf_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            
            if idz == 0:
                ax.fill_between(conf_area.lead_time,
                                -1 * conf_area, conf_area, color='gray', alpha=0.2, label='_nolegend_')
            ax.plot(sign_area.lead_time, sign_area, label=model_name[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()
        
        # Adjust y-axes limits to be symmetric -----
        ymin, ymax = ax.get_ylim()
        ylim = max([abs(ymin), abs(ymax)])
        ax.set_ylim(-ylim, ylim)

fig.tight_layout()

In [ ]:
linewidth = 2
fontsize = 12
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': fontsize})

anoms = ['c2']
masks = [1 + 0*where_elnino]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(6,4))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if (len(anoms) == 1) & (len(masks) == 1):
            ax = axes
        elif len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            sign_test = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                               model[anom].mean('ensemble').where(mask[anom]), 
                                               obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
            
            sign_masked = sign_test[0].where(mask[anom], drop=True)
            conf_masked = sign_test[1].where(mask[anom], drop=True)
            sign_area = doppyo.utils.integrate(sign_masked, over_dim='init_date', 
                                               x=(1+0*sign_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            conf_area = doppyo.utils.integrate(conf_masked, over_dim='init_date', 
                                               x=(1+0*conf_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            
            if idz == 0:
                ax.plot(conf_area.lead_time + 0.5, conf_area, 
                        color='grey', alpha=0.3, linestyle='--', label='_nolabel_')
                ax.plot(conf_area.lead_time + 0.5, -conf_area, 
                        color='grey', alpha=0.3, linestyle='--', label='_nolabel_')
                ax.fill_between(conf_area.lead_time + 0.5, -1 * conf_area, conf_area, 
                                facecolor='grey', alpha=0.2, linewidth=0.0)
            ax.plot(sign_area.lead_time + 0.5, sign_area, label=model_name[idz], linewidth=linewidth)
            
        if (idx == 0) & (idy == 0):
            legend = ax.legend(ncol=2, fontsize=fontsize-1)
            legend.get_frame().set_facecolor('w')
            legend.get_frame().set_edgecolor('w')
        
        # Adjust y-axes limits to be symmetric -----
        ymin, ymax = ax.get_ylim()
        ylim = max([abs(ymin), abs(ymax)])
        ax.set_ylim(-ylim, ylim)
        ax.set_xlim(0,12)
        ax.set_xlabel('lead [months]')
        ax.set_ylabel('Area under sign test')
        # ax.grid()
fig.tight_layout()

# How do the model **c1**, **c2** and **c1ncv** anomalies compare when assessed relative to observed **c2** anomalies?

In [ ]:
lead = 6
anoms = [('c1','c2'),('c1','c1ncv')]
masks = [1 + 0*where_elnino]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            sign, conf = doppyo.skill.sign_test(model[anom[0]].mean('ensemble').where(mask[anom[0]]), 
                                                model[anom[1]].mean('ensemble').where(mask[anom[1]]), 
                                                obs_2_use['c2'].sel(init_date=slice('1999','2015')).where(mask['c2']), scaled=False)
            if idz == 0:
                ax.fill_between(conf.init_date.values,
                                -1 * conf.sel(lead_time=lead), 
                                conf.sel(lead_time=lead), color='gray', alpha=0.2, label='_nolegend_')
            ax.plot(sign.init_date, sign.sel(lead_time=lead), label=model_name[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()

fig.tight_layout()

In [ ]:
anoms = [('c1','c3'),('c1','c1ncv')]
masks = [1 + 0*where_elnino]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            sign_test = doppyo.skill.sign_test(model[anom[0]].mean('ensemble').where(mask[anom[0]]), 
                                               model[anom[1]].mean('ensemble').where(mask[anom[1]]), 
                                               obs_2_use['c3'].sel(init_date=slice('1999','2015')).where(mask['c3']))
            
            sign_masked = sign_test[0].where(mask[anom[0]] & mask[anom[1]], drop=True)
            conf_masked = sign_test[1].where(mask[anom[0]] & mask[anom[1]], drop=True)
            sign_area = doppyo.utils.integrate(sign_masked, over_dim='init_date', 
                                               x=(1+0*sign_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            conf_area = doppyo.utils.integrate(conf_masked, over_dim='init_date', 
                                               x=(1+0*conf_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            
            if idz == 0:
                ax.fill_between(conf_area.lead_time,
                                -1 * conf_area, conf_area, color='gray', alpha=0.2, label='_nolegend_')
            ax.plot(sign_area.lead_time, sign_area, label=model_name[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()
        
        # Adjust y-axes limits to be symmetric -----
        ymin, ymax = ax.get_ylim()
        ylim = max([abs(ymin), abs(ymax)])
        ax.set_ylim(-ylim, ylim)

fig.tight_layout()

# What about **onset** of El Nino and La Nina?
Here we assess, for each of El Nino and La Nina, forecasts which end in an event, but are **not** started in the same event

In the first instance, observations are used to determine whether or not an event occurred. **This assesses the tendency of the forecasts to provide false negatives**

In [ ]:
where_elnino_onset = where_elnino.apply(where_condition_events, method='onset')
where_lanina_onset = where_lanina.apply(where_condition_events, method='onset')

In [ ]:
lead = 6
anoms = ['c0','c1','c2','c3']
masks = [where_elnino_onset,
         where_lanina_onset]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            sign, conf = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                                model[anom].mean('ensemble').where(mask[anom]), 
                                                obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
            
            if idz == 0:
                ax.fill_between(conf.init_date.values,
                                -1 * conf.sel(lead_time=lead), 
                                conf.sel(lead_time=lead), color='gray', alpha=0.2, label='_nolegend_')
            ax.plot(sign.init_date, sign.sel(lead_time=lead), label=model_name[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()

fig.tight_layout()

In [ ]:
anoms = ['c0','c1','c2','c3']
masks = [where_elnino_onset,
         where_lanina_onset]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            sign_test = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                               model[anom].mean('ensemble').where(mask[anom]), 
                                               obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
            
            sign_masked = sign_test[0].where(mask[anom]).fillna(0)
            conf_masked = sign_test[1].where(mask[anom]).fillna(0)
            sign_area = doppyo.utils.integrate(sign_masked, over_dim='init_date', 
                                               x=(1+0*sign_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            conf_area = doppyo.utils.integrate(conf_masked, over_dim='init_date', 
                                               x=(1+0*conf_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            
            if idz == 0:
                ax.fill_between(conf_area.lead_time,
                                -1 * conf_area, conf_area, color='gray', alpha=0.2, label='_nolegend_')
            ax.plot(sign_area.lead_time, sign_area, label=model_name[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()
        
        # Adjust y-axes limits to be symmetric -----
        ymin, ymax = ax.get_ylim()
        ylim = max([abs(ymin), abs(ymax)])
        ax.set_ylim(-ylim, ylim)

fig.tight_layout()

The above skill scores are conditioned on the actual occurence of ENSO events, and, as such, assess the tendency of the forecasts to provide false negatives. In the following, model forecasts are used to determine whether or not an event occurred. **This assesses the tendency of the forecasts to provide false positives.** As we shall see, these results prove very difficult to interpret because each model uses a very different subset of data in its assessment.

# Get forecast El Ninos and La Ninas

In [ ]:
where_elnino_onset_cola = where_elnino_cola.apply(where_condition_events, method='onset')
where_lanina_onset_cola = where_lanina_cola.apply(where_condition_events, method='onset')

where_elnino_onset_aer04 = where_elnino_aer04.apply(where_condition_events, method='onset')
where_lanina_onset_aer04 = where_lanina_aer04.apply(where_condition_events, method='onset')

where_elnino_onset_florA = where_elnino_florA.apply(where_condition_events, method='onset')
where_lanina_onset_florA = where_lanina_florA.apply(where_condition_events, method='onset')

where_elnino_onset_florB = where_elnino_florB.apply(where_condition_events, method='onset')
where_lanina_onset_florB = where_lanina_florB.apply(where_condition_events, method='onset')

where_elnino_onset_cm3 = where_elnino_cm3.apply(where_condition_events, method='onset')
where_lanina_onset_cm3 = where_lanina_cm3.apply(where_condition_events, method='onset')

where_elnino_onset_cm4 = where_elnino_cm4.apply(where_condition_events, method='onset')
where_lanina_onset_cm4 = where_lanina_cm4.apply(where_condition_events, method='onset')

In [ ]:
lead = 8
anoms = ['c0','c1','c2']
mask_strs = ['where_elnino_onset',
             'where_lanina_onset']
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_names = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(mask_strs), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask_str in enumerate(mask_strs):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            to_exec = 'mask = ' + mask_str + '_' + model_names[idz]
            exec(to_exec)
            sign, conf = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                               model[anom].mean('ensemble').where(mask[anom]), 
                                               obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
            #if idz == 0:
            ax.fill_between(conf.init_date.values,
                            -1 * conf.sel(lead_time=lead), 
                            conf.sel(lead_time=lead), alpha=0.2, label='_nolegend_')
            ax.plot(sign.init_date, sign.sel(lead_time=lead), label=model_names[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()

fig.tight_layout()

In [ ]:
anoms = ['c0','c1','c2']
mask_strs = ['where_elnino_onset',
             'where_lanina_onset']
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_names = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask_str in enumerate(mask_strs):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            to_exec = 'mask = ' + mask_str + '_' + model_names[idz]
            exec(to_exec)
            
            sign_test = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                               model[anom].mean('ensemble').where(mask[anom]), 
                                               obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
            
            sign_masked = sign_test[0].where(mask[anom]).fillna(0)
            conf_masked = sign_test[1].where(mask[anom]).fillna(0)
            sign_area = doppyo.utils.integrate(sign_masked, over_dim='init_date', 
                                               x=(1+0*sign_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            conf_area = doppyo.utils.integrate(conf_masked, over_dim='init_date', 
                                               x=(1+0*conf_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            
            # if idz == 0:
            ax.fill_between(conf_area.lead_time,
                            -1 * conf_area, conf_area, alpha=0.2, label='_nolegend_')
            ax.plot(sign_area.lead_time, sign_area, label=model_name[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()
        
        # Adjust y-axes limits to be symmetric -----
        ymin, ymax = ax.get_ylim()
        ylim = max([abs(ymin), abs(ymax)])
        ax.set_ylim(-ylim, ylim)

fig.tight_layout()

# What about **decay** of El Nino and La Nina?
Here we assess, for each of El Nino and La Nina, forecasts which end **outside of** an event, but are started in an event

Observations are used to determine whether or not an event occurred. **This assesses the tendency of the forecasts to provide false negatives**

In [ ]:
where_elnino_decay = where_elnino.apply(where_condition_events, method='decay')
where_lanina_decay = where_lanina.apply(where_condition_events, method='decay')

In [ ]:
lead = 6
anoms = ['c0','c1','c2','c3']
masks = [where_elnino_decay,
         where_lanina_decay]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            sign, conf = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                               model[anom].mean('ensemble').where(mask[anom]), 
                                               obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
            if idz == 0:
                ax.fill_between(conf.init_date.values,
                                -1 * conf.sel(lead_time=lead), 
                                conf.sel(lead_time=lead), color='gray', alpha=0.2, label='_nolegend_')
            ax.plot(sign.init_date, sign.sel(lead_time=lead), label=model_name[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()

fig.tight_layout()

In [ ]:
anoms = ['c0','c1','c2','c3']
masks = [where_elnino_decay,
         where_lanina_decay]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            sign_test = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                               model[anom].mean('ensemble').where(mask[anom]), 
                                               obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
            
            sign_masked = sign_test[0].where(mask[anom]).fillna(0)
            conf_masked = sign_test[1].where(mask[anom]).fillna(0)
            sign_area = doppyo.utils.integrate(sign_masked, over_dim='init_date', 
                                               x=(1+0*sign_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            conf_area = doppyo.utils.integrate(conf_masked, over_dim='init_date', 
                                               x=(1+0*conf_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            
            if idz == 0:
                ax.fill_between(conf_area.lead_time,
                                -1 * conf_area, conf_area, color='gray', alpha=0.2, label='_nolegend_')
            ax.plot(sign_area.lead_time, sign_area, label=model_name[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()
        
        # Adjust y-axes limits to be symmetric -----
        ymin, ymax = ax.get_ylim()
        ylim = max([abs(ymin), abs(ymax)])
        ax.set_ylim(-ylim, ylim)

fig.tight_layout()

# What about specifically decay **from** El Nino **to** La Nina?

In [ ]:
lead = 10
anoms = ['c0','c1','c2']
masks = [where_elnino_decay & where_lanina.where(where_lanina == 0, other=1)]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idx, idy]
        for idz, model in enumerate(models):
            sign_test = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                               model[anom].mean('ensemble').where(mask[anom]), 
                                               obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
            if idz == 0:
                ax.fill_between(sign_test[1].init_date.values,
                                -1 * sign_test[1].sel(lead_time=lead), 
                                sign_test[1].sel(lead_time=lead), color='gray', alpha=0.2, label='_nolegend_')
            ax.plot(sign_test[0].init_date, sign_test[0].sel(lead_time=lead), label=model_name[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()

fig.tight_layout()

In [ ]:
anoms = ['c0','c1','c2']
masks = [where_elnino_decay & where_lanina.where(where_lanina == 0, other=1)]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idx, idy]
        for idz, model in enumerate(models):
            sign_test = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                               model[anom].mean('ensemble').where(mask[anom]), 
                                               obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
            
            sign_area = doppyo.utils.integrate(sign_test[0], over_dim='init_date', 
                                               x=(1+0*sign_test[0].init_date.astype(int)).cumsum('init_date'))
            conf_area = doppyo.utils.integrate(sign_test[1], over_dim='init_date', 
                                               x=(1+0*sign_test[1].init_date.astype(int)).cumsum('init_date'))
            
            if idz == 0:
                ax.fill_between(conf_area.lead_time,
                                -1 * conf_area, conf_area, color='gray', alpha=0.2, label='_nolegend_')
            ax.plot(sign_area.lead_time, sign_area, label=model_name[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()
        
        # Adjust y-axes limits to be symmetric -----
        ymin, ymax = ax.get_ylim()
        ylim = max([abs(ymin), abs(ymax)])
        ax.set_ylim(-ylim, ylim)

fig.tight_layout()

# How do the results compare for c1 computed over the full 1982 -> 2015 period?

In [ ]:
lead = 3
anoms = ['c1','c1full']
masks = [1 + 0*where_elnino]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            sign, conf = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                               model[anom].mean('ensemble').where(mask[anom]), 
                                               obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
            if idz == 0:
                ax.fill_between(conf.init_date.values,
                                -1 * conf.sel(lead_time=lead), 
                                conf.sel(lead_time=lead), color='gray', alpha=0.2, label='_nolegend_')
            ax.plot(sign.init_date, sign.sel(lead_time=lead), label=model_name[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()

fig.tight_layout()

In [ ]:
anoms = ['c1','c1full']
masks = [1 + 0*where_elnino]
models = [cola_nino34, aer04_nino34, florA_nino34, florB_nino34, cm3_nino34, cm4_nino34]
model_name = ['cola','aer04','florA','florB','cm3','cm4']

fig = plt.figure(figsize=(5*len(anoms),3*len(masks)))
axes = fig.subplots(nrows=len(masks), ncols=len(anoms), sharex=True)
for idx, anom in enumerate(anoms):
    for idy, mask in enumerate(masks):
        if len(anoms) == 1:
            ax = axes[idy]
        elif len(masks) == 1:
            ax = axes[idx]
        else:
            ax = axes[idy, idx]
        for idz, model in enumerate(models):
            sign_test = doppyo.skill.sign_test(regr_nino34[anom].where(mask[anom]), 
                                               model[anom].mean('ensemble').where(mask[anom]), 
                                               obs_2_use[anom].sel(init_date=slice('1999','2015')).where(mask[anom]))
            
            sign_masked = sign_test[0].where(mask[anom], drop=True)
            conf_masked = sign_test[1].where(mask[anom], drop=True)
            sign_area = doppyo.utils.integrate(sign_masked, over_dim='init_date', 
                                               x=(1+0*sign_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            conf_area = doppyo.utils.integrate(conf_masked, over_dim='init_date', 
                                               x=(1+0*conf_masked.init_date.astype(int)).cumsum('init_date'), method='rect')
            
            if idz == 0:
                ax.fill_between(conf_area.lead_time,
                                -1 * conf_area, conf_area, color='gray', alpha=0.2, label='_nolegend_')
            ax.plot(sign_area.lead_time, sign_area, label=model_name[idz])
        if (idx == 0) & (idy == 0):
            ax.legend()
        
        # Adjust y-axes limits to be symmetric -----
        ymin, ymax = ax.get_ylim()
        ylim = max([abs(ymin), abs(ymax)])
        ax.set_ylim(-ylim, ylim)

fig.tight_layout()